# The TCS_MSL datamanager's toolbox

In [1]:
import infra, tcs_portal, os

## Generating laboratory information from Excel templates

### Identifiers

Initially we started assigning identifiers to labs in CKAN based on Damian Ulbricht's identifiersfile. Since the templates are going to play an essential role in the assignment of names the scenario has changed a bit.

_*New structure*_

- The assignment of an identifier remains based on the hash of a certain input string
- Formerly, this input string was built as 'name, (affiliation, country)'
- Because of the reality of possibly changing names, the input string no longer reflects any meaning (i.e. it does not longer reflect the preferred name by definition)
- Therefore, the centralized identifiersfile at GitLab will be comprised of the following JSON-keys:
    - 'domain'
    - 'inputstring' (fixed together with the identifier, also used for checking)
    - 'preferred name' (may change)
    - 'affiliation'
    - 'identifier'
- Algorithm in labinfo-extraction from Excel:
    - get name from **required** field name
    - if name in {idsfile:inputstring} get corresponding identifier
    - else get preferred name ('if other') [could be that this name has been used as inputstring in the past]
    - if preferred name in {idsfile:inputstring} get corresponding identifier
    - else: lab cannot be disambiguated to existing entry >> possibly new lab
    - if possibly new lab generate new id with inputstring {if exists pref name, else name}+(city, country)} [first check on instring city/country (pref-) name]

In [ ]:
# settings

working_dir = '/Users/otto/Documents/GitHub/TCS_MSL'

PALEOTEMPLATE = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Paleomag/Updated_Paleomag_Lab_Description_V6.2/TEMPLATE_Laboratory description_paleomagnetism_V6.2.xlsx'

PALEOFILES = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Paleomag/Updated_Paleomag_Lab_Description_V6.2/'

ROCKTEMPLATE = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Rock physics/Updated_Rock_Physics_Lab_Description_V3.2/TEMPLATE_Laboratory description_rock physics_V3.2.xlsx'

ROCKFILES = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Rock physics/Updated_Rock_Physics_Lab_Description_V3.2/'

ANALYTICALTEMPLATE = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Analytical&Microscopy/Updated_Analytical&Microscopy_Lab_Description_V3.1/TEMPLATE_Laboratory description_analytical labs_V3.1.xlsx'

ANALYTICALFILES = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Analytical&Microscopy/Updated_Analytical&Microscopy_Lab_Description_V3.1/'

ANALOGUETEMPLATE = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Analogue modelling/Updated_Analogue_Lab_Description_V3/TEMPLATE_Laboratory description_analogue modelling_V3.xlsx'

ANALOGUEFILES = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Analogue modelling/Updated_Analogue_Lab_Description_V3/'
    
IDENTIFIERSFILE = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/lab_identifiers.json'

JSONOUTPUT = '/Users/otto/Documents/GitLab/inframsl/labs.json'

JSONOUTPUT_ISSUES = '/Users/otto/Documents/GitLab/inframsl/labs_issues.json'

In [ ]:
infra.runConversion()

In [3]:
myCKAN = tcs_portal.TCS_PortalRequests()

In [4]:
ids = myCKAN.retrieveAllIdentifiers()

In [8]:
print(ids)

['abffc206fde4d92e6a99314edff00f98', '66dea222f20d276d6db8d51f29c6525f', 'c3fff0d9ca9fe9caae131ff2b11f6e67', 'f4b21c52c84ed94a290c0fe62ed8c647', '3b8fed72cf84e5c6bcccb0a74562a95d', 'e85bd300aa400f9bdb807912adb8c518', 'f9e0f3b0be7565fb3b95d1c2ece8978c', '9134453f876c393718c740124f7feaf3', 'e3a4f5d02528d02c516dbea19c20b32c', '9cd562c216daa82792972a074a222c52', '3a9d5ca4a33e7f95d0a2eaa1e8479a13', '6755019ede4f10b7393b127965236096', '3469c2abf3a0d3902c83e3adae683647', '5580650c9a079144208b62291de6e4fe', '02b7608a7e58bf889973a20d82d217fe', '1d216a0482566fbfb46c6677211968f4', 'f40f7fd30b6eb223669ffa5df445b85e', '009b51690050c9179ad1ff09fc8b5fdd', '9ba34c109b827b177aab36e0266b1643', '3f145405812f5c7f69206bbd78b33a86', '2613a7211e1498423ed604baff8efef9', '0d2892477f7ab2a23fdbbd22496e9d90', 'f28005bf8cf9ec0d284d123f0369131b', '5a9b77c98dbf6251c55f7074043e87fd', 'd0133f39c5081a996fa8edfc6bdbaade', 'f2973460d7af47a4cf1dd7853ffb03d7', 'c708c8a6df37eef7e827312f182ea80d', 'c8a8a396b310eb8b6aff45dc25

In [4]:
print(myCKAN.identifierInPortal('cdc8a61ba9f1327a72b75602773a8aa4'))

True


In [7]:
print(myCKAN.identifierInPortal('f780441f60a30a86b752e00adb66f107'))

True


**Managing individual Excel-file**

In [1]:
from labinfo.lab_info import labDescription as full_description
template_excel_wb = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Paleomag/Updated_Paleomag_Lab_Description_V6.2/New_Labs/TEMPLATE_Laboratory description_paleomagnetism_V6.3.xlsx'
lab_excel_wb = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Paleomag/Updated_Paleomag_Lab_Description_V6.2/New_Labs/Laboratory description_paleomagnetism_Liverpool_V6.3.xlsx'
research_field = 'Paleomagnetism'
subdomain = 'Paleomagnetic and magnetic data'
ids_file = '/Users/otto/ownCloud - EPOS/WP16/LABS description service/Lab info collected/Paleomag/Updated_Paleomag_Lab_Description_V6.2/New_Labs/labnames.json'
new_lab = full_description(lab_excel_wb, research_field, subdomain, template_excel_wb, ids_file)

Geomagnetism Laboratory (University of Liverpool, UK)
Geomagnetism Laboratory
Data publications: 0


In [2]:
print(new_lab.facility_info)

{'facility': {'type': 'laboratory', 'lab_id': '', 'research_infrastructure_name': 'University of Liverpool', 'facility_name': 'Geomagnetism Laboratory', 'address': {'street_with_number': 'Oliver Lodge Building, Oxford Street', 'postal_code': 'L69 7ZE', 'city': 'Liverpool', 'country': 'United Kingdom'}, 'gps': {'gps_lat': 53.3943, 'gps_lon': -3.0145}, 'website': 'https://www.liverpool.ac.uk/earth-ocean-and-ecological-sciences/research/earth-sciences/geomagnetism/', 'lab_services': {'research_field': 'Paleomagnetism', 'subdomain': ['Paleomagnetic and magnetic data'], 'equipment': [{'equipment_type': 'Rock_Magnetometer', 'equipment_group': 'Other', 'equipment_name': '', 'equipment_secondary_name': 'Tristan microwave palaeointensity system', 'equipment_brand': 'Built in-house', 'equipment_website': 'See facility website', 'equipment_specifics_and_comments': 'Integrated palaeointensity experiments (any Thellier protocol) on 5mm cylinders', 'equipment_quantity': 1.0, 'references': 'Suttie et